In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [4]:
print(train_df[train_df["target"]==0])
print(train_df[train_df["target"]==1])

         id  keyword location  \
15       23      NaN      NaN   
16       24      NaN      NaN   
17       25      NaN      NaN   
18       26      NaN      NaN   
19       28      NaN      NaN   
...     ...      ...      ...   
7581  10833  wrecked  Lincoln   
7582  10834  wrecked      NaN   
7584  10837      NaN      NaN   
7587  10841      NaN      NaN   
7593  10848      NaN      NaN   

                                                   text  target  
15                                       What's up man?       0  
16                                        I love fruits       0  
17                                     Summer is lovely       0  
18                                    My car is so fast       0  
19                         What a goooooooaaaaaal!!!!!!       0  
...                                                 ...     ...  
7581  @engineshed Great atmosphere at the British Li...       0  
7582  Cramer: Iger's 3 words that wrecked Disney's s...       0  
7584  The

Using the sklearn classes CountVectorizer and TfidfTransformer to create a training set using all rows of the ‘TEXT’ column from the train_df.

In [5]:
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(list(train_df["text"]))
print('Shape: ' + str(X_train_counts.shape) + '\n')
print('fire: '+ str(count_vect.vocabulary_.get('fire')) + '\n')
# print('All words in train_df: ' + str(count_vect.vocabulary_) + '\n')

Shape: (7613, 21637)

fire: 7439



In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf.shape)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(7613, 21637)
(7613, 21637)


Training and fitting a Multinomial Naive Bayes algorithm on the training data. The target variable is the ‘target’ column in train_df and test_df.

In [7]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_df['target'])

def find_the_language(docs_new):
    X_new_counts = count_vect.transform(docs_new)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    
    predicted = clf.predict(X_new_tfidf)
    return predicted
    
    # for doc, category in zip(docs_new, predicted):
    #     print('%r => %s' % (doc, category))

In [8]:
docs_new = [test_df['text'][0], test_df['text'][1]]
print(docs_new)
print(find_the_language(docs_new))

['Just happened a terrible car crash', 'Heard about #earthquake is different cities, stay safe everyone.']
[1 0]


In [9]:
docs_new = []
for sentence in test_df['text']:
  docs_new.append(sentence)

print(docs_new[0:10])

['Just happened a terrible car crash', 'Heard about #earthquake is different cities, stay safe everyone.', 'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all', 'Apocalypse lighting. #Spokane #wildfires', 'Typhoon Soudelor kills 28 in China and Taiwan', "We're shaking...It's an earthquake", "They'd probably still show more life than Arsenal did yesterday, eh? EH?", 'Hey! How are you?', 'What a nice hat?', 'Fuck off!']


In [10]:
find_the_language(docs_new)

array([1, 0, 1, ..., 1, 1, 1])

In [11]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [12]:
sample_submission["target"] = find_the_language(docs_new)
sample_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [13]:
sample_submission.to_csv("submission.csv", index=False)